# Analyzing data from Task 1 

In [4]:
import pandas as pd
import polars as pl 

In [8]:
# import os
# os.getcwd()

In [7]:
task1_df = pl.read_csv("../data/processed/task1.csv")
task1_df

,participantId,task,id,rsp_time,status,data.select.x,data.select.y,pixel.select.x,pixel.select.y,param.mu,param.sigma,param.lambda,param.p,param.q,pixelToMM,dist_to_screen,data.select.left_area,data.ans.x,pixel.med.x,pixel.mod.x,phy.select.x,phy.med.x,phy.mod.x,va.select.x,va.med.x,va.mod.x
i64,str,str,i64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""67f03733938aa3f0d17ce8a8""","""task1""",1,8.22,"""completed""",0.34,0.218606,335.69,323.650744,0.41,1.5,-0.01,3.69,16.55,4.6,668.258963,"""0.489697578893479""",0.387128,338.211338,339.435,62.106522,62.654639,62.920652,5.321116,5.368009,5.390766
2,"""67f03733938aa3f0d17ce8a8""","""task1""",2,5.467,"""completed""",0.09,0.310524,322.315,287.34312,-0.65,1.17,0.53,2.92,5.89,4.6,668.258963,"""0.4692395323358""",0.189605,327.643861,282.725,59.198913,60.357361,50.592391,5.072333,5.171459,4.335665
3,"""67f03733938aa3f0d17ce8a8""","""task1""",3,5.354,"""completed""",0.34,0.521065,335.69,204.179468,0.42,0.62,-0.15,3.94,37.73,4.6,668.258963,"""0.53331408923348""",0.276061,332.26924,339.97,62.106522,61.362878,63.036957,5.321116,5.257493,5.400716
4,"""67f03733938aa3f0d17ce8a8""","""task1""",4,5.059,"""completed""",0.09,0.350077,322.315,271.719768,-0.89,1.05,0.79,2.6,27.22,4.6,668.258963,"""0.467448266910563""",0.183909,327.339129,269.885,59.198913,60.291115,47.801087,5.072333,5.165791,4.096666
5,"""67f03733938aa3f0d17ce8a8""","""task1""",5,5.606,"""completed""",-0.43,0.634068,294.495,159.543153,-0.37,0.51,-0.15,3.91,46.15,4.6,668.258963,"""0.536955412831699""",-0.488288,291.376613,297.705,53.151087,52.473177,53.848913,4.554715,4.496682,4.61445
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
156,"""67ee2686d7cfcbbd9d87dd04""","""task1""",6,16.482,"""completed""",1.26,0.478192,384.91,221.11416,1.71,0.71,-0.81,3.97,37.05,4.482353,587.915846,"""0.689415294700752""",0.860117,363.516276,408.985,74.717454,69.944576,80.088517,7.271868,6.808478,7.79305
157,"""67ee2686d7cfcbbd9d87dd04""","""task1""",7,66.044,"""completed""",1.55,0.280573,400.425,299.173578,1.95,1.33,-0.46,2.54,38.2,4.482353,587.915846,"""0.617239609280585""",1.126147,377.748891,421.825,78.178806,73.119831,82.953084,7.607777,7.116784,8.070883
158,"""67ee2686d7cfcbbd9d87dd04""","""task1""",8,46.364,"""completed""",1.42,0.439767,393.47,236.291984,1.42,0.72,0.05,4.4,28.29,4.482353,587.915846,"""NA""",1.476848,396.511383,393.47,76.627165,77.305689,76.627165,7.457213,7.523057,7.457213


In [10]:
type(task1_df)

polars.dataframe.frame.DataFrame

# Comiple the stan model 

... using the `stan` file. 

Following guide [here](https://mc-stan.org/cmdstanpy/users-guide/workflow.html). 

In [15]:
import os 
from cmdstanpy import CmdStanModel

task1_stanfile = os.path.join('../', "analysis/final/stan_files/task1.weighted.avg.nonc.stan")
task1_model = CmdStanModel(stan_file=task1_stanfile)

15:41:23 - cmdstanpy - INFO - compiling stan file /Users/shenglong/Downloads/vis-decode-analysis/analysis/final/stan_files/task1.weighted.avg.nonc.stan to exe file /Users/shenglong/Downloads/vis-decode-analysis/analysis/final/stan_files/task1.weighted.avg.nonc
15:41:32 - cmdstanpy - INFO - compiled model executable: /Users/shenglong/Downloads/vis-decode-analysis/analysis/final/stan_files/task1.weighted.avg.nonc


In [21]:
# task1_model.name
# task1_model.stan_file
# task1_model.exe_file
print(task1_model.code())

data {
  int<lower=1> N;   // total number of observations
  int<lower=1> J;   // total number of participants 
  array[N] int<lower=1, upper=J> id; // participant ID for each observation 
  vector[N] x;      // observed values
  vector[N] x_med;  // known median values
  vector[N] x_mod;  // known mode values
}

parameters {
  // Population-level parameters 
  real mu_mod_mean;
  real<lower=0> mu_mod_sd; 
  real mu_med_mean;
  real<lower=0> mu_med_sd; 
  
  // standard deviations for x_med and x_mod
  real log_sigma_mod_mean; 
  real<lower=0> log_sigma_mod_sd; 
  real log_sigma_med_mean; 
  real<lower=0> log_sigma_med_sd;

  vector[J] mu_mod_z; 
  vector[J] mu_med_z; 
  vector[J] log_sigma_mod_z; 
  vector[J] log_sigma_med_z; 
}

transformed parameters {
  // Particiipant-level parameters
  vector[J] mu_mod; 
  vector<lower=0>[J] sigma_mod; 
  vector[J] mu_med; 
  vector<lower=0>[J] sigma_med; 

  // Non-centered param 
  mu_mod = mu_mod_mean + mu_mod_z * mu_mod_sd; 
  sigma_mod = exp